<a href="https://colab.research.google.com/github/reneholt/buffer-overflow-stack/blob/main/Buffer_overflow_stack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What are buffer overflow attacks and how are they thwarted?

**Example – Encoding hexadecimal characters as byte values**

For software developers interested in a recent buffer overflow discovered in 2021, we offer the following code in C, which is a simplified and rewritten version of a vulnerability in the ZTE MF971R LTE router tracked as CVE‑2021‑21748:

In [19]:
%%writefile EncodeHex.c

#include <stdio.h>
#include <string.h>

void encodeHexAsByteValues(const char *hexString, char *byteValues) {
	signed int offset = 0;
	int hexValues = 0;
	int byte = 0;

	while (offset < strlen(hexString)) {
        strncpy((char *) &hexValues, &hexString[2 * offset], 2u);
        sscanf((const char *) &hexValues, "%02X", &byte);
        byteValues[offset++] = byte;       // The return address can be overwritten opening a path for the 
										                      // insertion of exploit code
    }
}

int main(void) {
		const char* hexString = "0123456789ABCDEF01234";

		char byteValues[4];       

		encodeHexAsByteValues(hexString, byteValues);	// There is no size check to ensure that   
														                      // hexString is not too long for byteValues 
													                      	// before calling the function
		return 0;
}

Overwriting EncodeHex.c


In [20]:
%%shell
gcc EncodeHex.c -o EncodeHex
./EncodeHex

*** stack smashing detected ***: <unknown> terminated


CalledProcessError: ignored

The program above demonstrates a function that encodes a string consisting of hexadecimal-compatible characters into a form with half the memory requirement. Two characters can stand in as actual byte values (in hexadecimal), so that the characters ‘0’ and ‘1’, represented with the byte values 30 and 31, respectively, can be represented literally as the byte value 01. This functionality was used as part of the ZTE router’s handling of passwords.

As noted in the comments of the code, the hexString, having a size of 21 characters, is too large for the byteValues buffer, which only has a size of 4 characters (even though it can accept up to 8 characters in encoded form), and there is no check to ensure that the encodeHexAsByteValues function won’t lead to a buffer overflow.